In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
#Data Extracted from The COVID Tracking Project (as of 4/8/2020)
#https://covidtracking.com/
import pandas as pd
import numpy as np
import requests
import json
import datetime
!pip install pycountry
import pycountry
!pip install 'wget'
import wget
import datetime

     |████████████████████████████████| 10.0MB 9.3MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/a2/98/bf/f0fa1c6bf8cf2cbdb750d583f84be51c2cd8272460b8b36bd3
Successfully built pycountry
  Stored in directory: /home/dsxuser/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [3]:
raw_response = requests.get("https://covidtracking.com/api/states/daily").text
raw_data = pd.DataFrame.from_dict(json.loads(raw_response))

In [4]:
data = raw_data.fillna(0)
data['Date'] = pd.to_datetime(data['date'].astype(str), format='%Y%m%d')
data = data.rename(
    columns={
        "state": "ISO3166-2",
        "positive": "Positive",
        "negative": "Negative",
        "pending": "Pending",
        "death": "Death",
        "totalTestResults": "Total",
        "hospitalized": "Hospitalized"
    })
data = data.drop(labels=['dateChecked', "date"], axis='columns')
data['Country/Region'] = "United States"
data['ISO3166-1'] = "US"

In [5]:
states = {k.code.replace("US-", ""): k.name for k in pycountry.subdivisions.get(country_code="US")}

In [6]:
data["Province/State"] = data["ISO3166-2"].apply(lambda x: states[x])

In [7]:
sorted_data = data.sort_values(by=['Province/State'] + ['Date'], ascending=True)

In [8]:
sorted_data['Current_Day_Positive_Cases'] = sorted_data['Positive'] - sorted_data.groupby(['Province/State'])["Positive"].shift(1, fill_value=0)
sorted_data['Current_Day_Total_Tests'] = sorted_data['Total'] - sorted_data.groupby(['Province/State'])["Total"].shift(1, fill_value=0)
sorted_data['Current_Day_Negative_Cases'] = sorted_data['Negative'] - sorted_data.groupby(['Province/State'])["Negative"].shift(1, fill_value=0)
sorted_data['Current_Day_Pending_Cases'] = sorted_data['Pending'] - sorted_data.groupby(['Province/State'])["Pending"].shift(1, fill_value=0)
sorted_data['Current_Day_Deaths'] = sorted_data['Death'] - sorted_data.groupby(['Province/State'])["Death"].shift(1, fill_value=0)
sorted_data['Current_Day_Hospitalized'] = sorted_data['Hospitalized'] - sorted_data.groupby(['Province/State'])["Hospitalized"].shift(1, fill_value=0)
sorted_data['Prior_Day_Positive_Cases'] = sorted_data.groupby('Province/State')['Positive'].shift(1, fill_value=0)
sorted_data['Prior_Day_Deaths'] = sorted_data.groupby('Province/State')['Death'].shift(1, fill_value=0)
sorted_data['Prior_Day_Total_Tests'] = sorted_data.groupby('Province/State')['Total'].shift(1, fill_value=0)
sorted_data['Pct_Change_Positive_Cases'] = (sorted_data['Positive'] - sorted_data['Prior_Day_Positive_Cases'])/sorted_data['Prior_Day_Positive_Cases']
sorted_data['Pct_Change_Deaths'] = (sorted_data['Death'] - sorted_data['Prior_Day_Deaths'])/sorted_data['Prior_Day_Deaths']
sorted_data['Pct_Change_Total_Tests'] = (sorted_data['Total'] - sorted_data['Prior_Day_Total_Tests'])/sorted_data['Prior_Day_Total_Tests']
sorted_data['Cumulative_Positive_Cases'] = sorted_data['Positive']
sorted_data['Cumulative_Total_Tests'] = sorted_data['Total']
sorted_data['Cumulative_Deaths'] = sorted_data['Death']

In [9]:
#bring in csv for state population estimates
urls = ['https://raw.githubusercontent.com/rdewolf127/DataVisualization611/master/state_population_est_2019.csv']
for url in urls:
    filename = wget.download(url)
filename

'state_population_est_2019.csv'

In [10]:
state_pop = pd.read_csv('state_population_est_2019.csv', dtype={'fips':'str'})
state_pop.head()

,fips,Province/State,PopEst_2019,Per_500K_Factor
0,01,.Alabama,"4,903,185",9.806370
1,02,.Alaska,"731,545",1.463090
2,04,.Arizona,"7,278,717",14.557434
3,05,.Arkansas,"3,017,804",6.035608
4,06,.California,"39,512,223",79.024446


In [11]:
#bring in csv for state population estimates
urls = ['https://raw.githubusercontent.com/rdewolf127/DataVisualization611/master/state_population_density.csv']
for url in urls:
    filename = wget.download(url)
filename

'state_population_density.csv'

In [12]:
state_density = pd.read_csv('state_population_density.csv', dtype={'fips':'str'})
state_density.head()

,fips,state,landareasquaremiles,popdensity19
0,01,Alabama,50645.33,96.81
1,02,Alaska,553559.49,1.32
2,04,Arizona,113594.09,64.08
3,05,Arkansas,52035.44,58.00
4,06,California,155779.19,253.64


In [13]:
merged_data = pd.merge(sorted_data, state_pop, on='fips')
merged_data = pd.merge(merged_data, state_density, on='fips')
merged_data['Province/State']=merged_data['Province/State_x']
merged_data.head()

,Death,deathIncrease,fips,hash,Hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,inIcuCumulative,inIcuCurrently,...,Cumulative_Positive_Cases,Cumulative_Total_Tests,Cumulative_Deaths,Province/State_y,PopEst_2019,Per_500K_Factor,state,landareasquaremiles,popdensity19,Province/State
0,0.0,0.0,01,1d94a3b00af81e57b35a00c0eeaa339ec76d962a,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,.Alabama,"4,903,185",9.80637,Alabama,50645.33,96.81,Alabama
1,0.0,0.0,01,dc8486565b986e301a3cb52dda7d6014007253e4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,.Alabama,"4,903,185",9.80637,Alabama,50645.33,96.81,Alabama
2,0.0,0.0,01,3c15dbef6838aa05f46554b8f49c5d96677df531,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,.Alabama,"4,903,185",9.80637,Alabama,50645.33,96.81,Alabama
3,0.0,0.0,01,d77d1bec988a8d98086652dbe2ec43647b585934,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,.Alabama,"4,903,185",9.80637,Alabama,50645.33,96.81,Alabama
4,0.0,0.0,01,e4593577708b6182b9126434908df283def67838,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,.Alabama,"4,903,185",9.80637,Alabama,50645.33,96.81,Alabama


In [14]:
rearranged_data = merged_data.filter(items=['fips','Country/Region', 'Province/State', 'Date',
                               'Cumulative_Positive_Cases', 'Current_Day_Positive_Cases',
                               'Prior_Day_Positive_Cases','Pct_Change_Positive_Cases',
                               'Cumulative_Total_Tests', 'Current_Day_Total_Tests',
                               'Prior_Day_Total_Tests', 'Pct_Change_Total_Tests',
                               'Cumulative_Deaths', 'Current_Day_Deaths',
                               'Prior_Day_Deaths', 'Pct_Change_Deaths',
                               'ISO3166-1', 'ISO3166-2', 'PopEst_2019', 'Per_500K_Factor','landareasquaremiles', 'popdensity19'])

In [15]:
rearranged_data.loc[:, "Last_Update_Date"] = datetime.datetime.utcnow()
rearranged_data.head()

,fips,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,...,Current_Day_Deaths,Prior_Day_Deaths,Pct_Change_Deaths,ISO3166-1,ISO3166-2,PopEst_2019,Per_500K_Factor,landareasquaremiles,popdensity19,Last_Update_Date
0,01,United States,Alabama,2020-03-07,0.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.0,NaN,US,AL,"4,903,185",9.80637,50645.33,96.81,2020-04-23 21:57:08.261323
1,01,United States,Alabama,2020-03-08,0.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.0,NaN,US,AL,"4,903,185",9.80637,50645.33,96.81,2020-04-23 21:57:08.261323
2,01,United States,Alabama,2020-03-09,0.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.0,NaN,US,AL,"4,903,185",9.80637,50645.33,96.81,2020-04-23 21:57:08.261323
3,01,United States,Alabama,2020-03-10,0.0,0.0,0.0,NaN,0.0,0.0,...,0.0,0.0,NaN,US,AL,"4,903,185",9.80637,50645.33,96.81,2020-04-23 21:57:08.261323
4,01,United States,Alabama,2020-03-11,0.0,0.0,0.0,NaN,10.0,10.0,...,0.0,0.0,NaN,US,AL,"4,903,185",9.80637,50645.33,96.81,2020-04-23 21:57:08.261323


In [16]:
rearranged_data['Cumulative_Mortality_Rate']=rearranged_data['Cumulative_Deaths']/rearranged_data['Cumulative_Positive_Cases']
rearranged_data['Cumulative_Positive_Test_Rate'] = rearranged_data['Cumulative_Positive_Cases']/rearranged_data['Cumulative_Total_Tests']
rearranged_data['Current_Day_Mortality_Rate']=rearranged_data['Current_Day_Deaths']/rearranged_data['Current_Day_Positive_Cases']
rearranged_data['Current_Day_Positive_Test_Rate'] = rearranged_data['Current_Day_Positive_Cases']/rearranged_data['Current_Day_Total_Tests']
rearranged_data['Cumulative_Positive_Cases_Per_500K'] = round(rearranged_data['Cumulative_Positive_Cases']/rearranged_data['Per_500K_Factor'],0)
rearranged_data['Cumulative_Positive_Cases_Per_500K - LOG'] = np.log(rearranged_data['Cumulative_Positive_Cases_Per_500K'])
rearranged_data['popdensity19 - LOG'] = np.log(rearranged_data['popdensity19'])
rearranged_data.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: divide by zero encountered in log


,fips,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,...,landareasquaremiles,popdensity19,Last_Update_Date,Cumulative_Mortality_Rate,Cumulative_Positive_Test_Rate,Current_Day_Mortality_Rate,Current_Day_Positive_Test_Rate,Cumulative_Positive_Cases_Per_500K,Cumulative_Positive_Cases_Per_500K - LOG,popdensity19 - LOG
0,01,United States,Alabama,2020-03-07,0.0,0.0,0.0,NaN,0.0,0.0,...,50645.33,96.81,2020-04-23 21:57:08.261323,NaN,NaN,NaN,NaN,0.0,-inf,4.57275
1,01,United States,Alabama,2020-03-08,0.0,0.0,0.0,NaN,0.0,0.0,...,50645.33,96.81,2020-04-23 21:57:08.261323,NaN,NaN,NaN,NaN,0.0,-inf,4.57275
2,01,United States,Alabama,2020-03-09,0.0,0.0,0.0,NaN,0.0,0.0,...,50645.33,96.81,2020-04-23 21:57:08.261323,NaN,NaN,NaN,NaN,0.0,-inf,4.57275
3,01,United States,Alabama,2020-03-10,0.0,0.0,0.0,NaN,0.0,0.0,...,50645.33,96.81,2020-04-23 21:57:08.261323,NaN,NaN,NaN,NaN,0.0,-inf,4.57275
4,01,United States,Alabama,2020-03-11,0.0,0.0,0.0,NaN,10.0,10.0,...,50645.33,96.81,2020-04-23 21:57:08.261323,NaN,0.0,NaN,0.0,0.0,-inf,4.57275


In [17]:
rearranged_data['MA7_Pct_Change_Positive_Cases'] = rearranged_data.rolling(window=7)['Pct_Change_Positive_Cases'].mean()
rearranged_data['MA7_Pct_Change_Total_Tests'] = rearranged_data.rolling(window=7)['Pct_Change_Total_Tests'].mean()
rearranged_data['MA7_Pct_Change_Deaths'] = rearranged_data.rolling(window=7)['Pct_Change_Deaths'].mean()
rearranged_data['MA7_Cumulative_Mortality_Rate'] = rearranged_data.rolling(window=7)['Cumulative_Mortality_Rate'].mean()
rearranged_data['MA7_Current_Day_Mortality_Rate'] = rearranged_data.rolling(window=7)['Current_Day_Mortality_Rate'].mean()
rearranged_data['MA7_Cumulative_Positive_Test_Rate'] = rearranged_data.rolling(window=7)['Cumulative_Positive_Test_Rate'].mean()
rearranged_data['MA7_Current_Day_Positive_Test_Rate'] = rearranged_data.rolling(window=7)['Current_Day_Positive_Test_Rate'].mean()
rearranged_data['MA7_Current_Day_Positive_Cases'] = rearranged_data.rolling(window=7)['Current_Day_Positive_Cases'].mean()
rearranged_data.head()

,fips,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,...,Cumulative_Positive_Cases_Per_500K - LOG,popdensity19 - LOG,MA7_Pct_Change_Positive_Cases,MA7_Pct_Change_Total_Tests,MA7_Pct_Change_Deaths,MA7_Cumulative_Mortality_Rate,MA7_Current_Day_Mortality_Rate,MA7_Cumulative_Positive_Test_Rate,MA7_Current_Day_Positive_Test_Rate,MA7_Current_Day_Positive_Cases
0,01,United States,Alabama,2020-03-07,0.0,0.0,0.0,NaN,0.0,0.0,...,-inf,4.57275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,United States,Alabama,2020-03-08,0.0,0.0,0.0,NaN,0.0,0.0,...,-inf,4.57275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,United States,Alabama,2020-03-09,0.0,0.0,0.0,NaN,0.0,0.0,...,-inf,4.57275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,United States,Alabama,2020-03-10,0.0,0.0,0.0,NaN,0.0,0.0,...,-inf,4.57275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,United States,Alabama,2020-03-11,0.0,0.0,0.0,NaN,10.0,10.0,...,-inf,4.57275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
MA_only = rearranged_data[(rearranged_data['Province/State']=='Massachusetts')]
MA_only.tail()

,fips,Country/Region,Province/State,Date,Cumulative_Positive_Cases,Current_Day_Positive_Cases,Prior_Day_Positive_Cases,Pct_Change_Positive_Cases,Cumulative_Total_Tests,Current_Day_Total_Tests,...,Cumulative_Positive_Cases_Per_500K - LOG,popdensity19 - LOG,MA7_Pct_Change_Positive_Cases,MA7_Pct_Change_Total_Tests,MA7_Pct_Change_Deaths,MA7_Cumulative_Mortality_Rate,MA7_Current_Day_Mortality_Rate,MA7_Cumulative_Positive_Test_Rate,MA7_Current_Day_Positive_Test_Rate,MA7_Current_Day_Positive_Cases
1082,25,United States,Massachusetts,2020-04-19,38077.0,1705.0,36372.0,0.046877,162241.0,5435.0,...,7.923710,6.784061,0.059142,0.048208,0.123447,0.038517,0.076199,0.227975,0.280804,1800.285714
1083,25,United States,Massachusetts,2020-04-20,38077.0,0.0,38077.0,0.000000,162241.0,0.0,...,7.923710,6.784061,0.051336,0.041698,0.106818,0.040430,NaN,0.230055,NaN,1601.428571
1084,25,United States,Massachusetts,2020-04-21,41199.0,3122.0,38077.0,0.081992,175372.0,13131.0,...,8.002694,6.784061,0.056158,0.047991,0.109045,0.042376,NaN,0.231823,NaN,1862.285714
1085,25,United States,Massachusetts,2020-04-22,42944.0,1745.0,41199.0,0.042355,180462.0,5090.0,...,8.043984,6.784061,0.053306,0.045960,0.102604,0.044344,NaN,0.233446,NaN,1860.857143
1086,25,United States,Massachusetts,2020-04-23,46023.0,3079.0,42944.0,0.071698,195076.0,14614.0,...,8.113427,6.784061,0.052743,0.048061,0.096594,0.046142,NaN,0.234492,NaN,1977.428571


In [19]:
date=datetime.date.today().isoformat()
date
date_csv_all = 'all_us_'+date+'.csv'
date_csv_all
date_csv_ma = 'ma_only_'+date+'.csv'
date_csv_ma

'ma_only_2020-04-23.csv'

In [20]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'all_us_2020-04-23.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'covid19-donotdelete-pr-ysdyv5czxoifrj',
 'asset_id': '04f0d22f-e987-48da-81d8-9168cb3dfa67'}

In [21]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'ma_only_2020-04-23.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'covid19-donotdelete-pr-ysdyv5czxoifrj',
 'asset_id': '3e578e9b-0c56-4592-9eca-4ec160d7fd8d'}